In [ ]:
# strip out all @domain.com 
perl -pe 's/^(.*?)@.*,(.*)$/\1,\2/g' comb_out.csv > comb_username_pass.csv

# sort by username and get count
cut -d ',' -f 1  comb_username_pass.csv | sort | uniq -c | sort -nr > sorted_users.txt

# get usernames which repeated more than 100
awk '$1>100' sorted_users.txt > sorted_users_more_100.txt

# get those usernames to seperate file
awk '{print $2}' sorted_users_more_100.txt > sorted_users_more_100_only_names.txt

# create regex for these user names so we get exact match 
sed -i 's/\(^.*$\)/^\\b\1,\\b/g' sorted_users_more_100_only_names.txt

# chunk into multiple files (or else causing memory issues)
split -l 10000 sorted_users_more_100_only_names.txt chunked

# this is to make sure that we remove any binary data
strings comb_username_pass.csv > user_pass_clean.csv

# now search using each chunk file
for i in chunked*; do
grep -E -f "$i" user_pass_clean.csv >> final_users_more_100.csv
rm "$i"
done

# some usernames have comma in it so clean them up
awk -F ',' 'NF==2' final_users_more_100.csv > final_users_more_100_clean.csv

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your data (replace 'final_users_more_100.csv' with your file path)
df = pd.read_csv('data/final_users_more_100_clean.csv', names=['username','pass'], on_bad_lines="skip")

In [14]:
df.shape

(277488354, 2)

In [15]:
df.head()

,username,pass
0,1001aa,1001aa
1,1001aa,1001aa
2,1001aa,1001aa
3,1001aa,cfrfvenbrf
4,1001aa,chfrfvenbrf


In [17]:
df = df.dropna()

In [18]:
df.shape

(277471690, 2)

In [20]:
# Count the occurrences of each username
counts = df['username'].value_counts()

In [21]:
# Get a boolean Series indicating which rows have a username that appears at least 3 times
filter = df['username'].isin(counts[counts >= 3].index)

In [22]:
# Keep only the rows with a username that appears at least 2 times
df = df[filter]

In [23]:
df.shape

(277253930, 2)

In [25]:
# Split data into train + validation and test sets (90% vs 10%)
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Split train_val_df into separate training and validation sets (80% vs 20% of initial dataframe)
train_df, val_df = train_test_split(train_val_df, test_size=0.1111, random_state=42)

# Now we have train_df (80% of data), val_df (10% of data) and test_df (10% of data)

In [26]:
print("done")

done


In [27]:
# Save them to CSV files
train_df.to_csv('data/train_100.csv', index=False)
val_df.to_csv('data/val_100.csv', index=False)
test_df.to_csv('data/test_100.csv', index=False)

In [28]:
train_df.shape

(221805916, 2)

In [29]:
val_df.shape

(27722621, 2)

In [30]:
test_df.shape

(27725393, 2)